In [ ]:
#!usr/bin/env python
# encoding:utf-8
from __future__ import division
 
 
"""
__Author__:沂水寒城
功能： yaml 操作
"""
 
 
import sys
import yaml
 
 
def write2Yaml(data, save_path="test.yaml"):
    """
    存储yaml文件
    """
    with open(save_path, "w") as f:
        yaml.dump(data, f)
 
 
def loadData(data="config.yaml"):
    """
    加载yaml文件
    """
    with open(data, "r") as f:
        content = f.read()
    yamlData = yaml.load(content)
    print("yamlData_type: ", type(yamlData))
    print("yamlData: ", yamlData)
    return yamlData
 
 
if __name__ == "__main__":
    data = {
        "kind": "SeldonDeployment",
        "spec": {
            "name": "test-deployment",
            "predictors": [
                {
                    "graph": {
                        "endpoint": {"type": "REST"},
                        "type": "MODEL",
                        "name": "step_one",
                        "children": {
                            "endpoint": {"type": "REST"},
                            "type": "MODEL",
                            "name": "step_two",
                            "children": {
                                "endpoint": {"type": "REST"},
                                "type": "MODEL",
                                "name": "step_three",
                                "children": [],
                            },
                        },
                    },
                    "componentSpecs": [
                        {
                            "spec": {
                                "containers": [
                                    {
                                        "image": "seldonio/step_one:1.0",
                                        "name": "step_one",
                                    },
                                    {
                                        "image": "seldonio/step_two:1.0",
                                        "name": "step_two",
                                    },
                                    {
                                        "image": "seldonio/step_three:1.0",
                                        "name": "step_three",
                                    },
                                ]
                            }
                        }
                    ],
                    "name": "example",
                    "replicas": 1,
                }
            ],
        },
        "apiVersion": "machinelearning.seldon.io/v1alpha2",
        "metadata": {"name": "seldon-model"},
    }
 
 
    write2Yaml(data, save_path="test.yaml")
 
    yamlData = loadData(data="test.yaml")
 
 
    print(yamlData == data)
 

In [ ]:
import pandas as pd
import datetime

time_step = 1/60
start_dt = datetime.datetime.strptime("2021-05-25 00:01","%Y-%m-%d %H:%M")
end_dt = datetime.datetime.strptime("2021-06-29 00:00","%Y-%m-%d %H:%M")
time_delta = end_dt - start_dt
total_min = time_delta.days *24 *60 + time_delta.seconds // 60 + 1
step_per_min = 1/time_step/60
time_num = total_min *step_per_min
print(total_min)


dates = pd.date_range(start=start_dt,end=end_dt,freq='min')
date_list = [x.strftime("%H:%M") for x in dates]
indice_num = len(dates)//4
show_dates = date_list[::indice_num] + [date_list[-1]]


show_dates = [show_dates[i].split(" ")[-1] for i in range(4)]
print(show_dates)

timestamp = str(datetime.datetime.now()).split(".")[0].replace(" ", "T").replace("-","").replace(":","")
print(timestamp)

import os
p = os.path.split(os.path.realpath(__file__))[0]
print(p)

# 面源污染径流产生过程线模型

In [1]:
## 面源污染负荷估算主程序
import os
from datetime import datetime
import pandas as pd
import xlrd
import numpy as np

def get_month(begin,end):
    begin_year,end_year=begin.year,end.year
    begin_month,end_month=begin.month,end.month
    if begin_year==end_year:
        months=end_month-begin_month
    else:
        months=(end_year-begin_year)*12+end_month-begin_month
    return months + 1


""" 1. 读取Project文件"""
file_path = 'data/non_point_pollution/Project.xlsx'
io = pd.io.excel.ExcelFile(file_path)
timeseriesdata = pd.read_excel(io,sheet_name="TimeSeries").values
patchdata = pd.read_excel(io,sheet_name="Patch").values
pollutantdata = pd.read_excel(io,sheet_name="Pollutant").values
controlunitdata = pd.read_excel(io,sheet_name="ControlUnit").values
ludata = pd.read_excel(io,sheet_name="LandUse").values
usdata = pd.read_excel(io,sheet_name="UnderlyingSurface").values
io.close()

# 读取时间、降雨量、蒸发量数据
tlabel = [xlrd.xldate.xldate_as_datetime(t,0) 
          for t in timeseriesdata[:, 0]]
rain = timeseriesdata[:, 1]
evap = timeseriesdata[:, 2]

nhour = len(tlabel)
nday = int((tlabel[-1]-tlabel[0]).days) + 1
nmonth = get_month(tlabel[0],tlabel[-1])
nyear = 1

# 读取整个城区所有斑块的id数据
patchid = patchdata[:, 0]
patchstreet = patchdata[:, 1]
patchlu = patchdata[:, 2]
patchus = patchdata[:, 3]
patcharea = patchdata[:, 4]
N = len(patchid)

# 读取污染指标对应的id
pollutantid = pollutantdata[:, 0]
npl = len(pollutantid)

# 读取控制单元对应的街道和区的id
streetid = controlunitdata[:, 0]
streetdistrict = controlunitdata[:, 1]
ns = len(streetid)
nd = max(streetdistrict)

# 读取土地利用类型的id
luid = ludata[:, 0]
nlu = len(luid)

# 读取下垫面类型的id以及对应的最大初损、下渗率
usid = usdata[:, 0]
usloss = usdata[:, 2]
usinfil = usdata[:, 3]
nus = len(usid)

# 读取所有土地利用类型和下垫面类型对应的水质参数
P = {p:{"Bmax":None,
        "bt":None,
        "C1":None,
        "C2":None,
        "EMC":None,} 
     for p in range(npl)}
for p in range(npl):
    P[p]["Bmax"] = ludata[:, 2+p] ##ok<*SAGROW>
    P[p]["bt"] = ludata[:, 2+npl+p]
    P[p]["C1"] = usdata[:, 4+p]
    P[p]["C2"] = usdata[:, 4+npl+p]
    P[p]["EMC"] = usdata[:, 4+2*npl+p]


"""2. 单位面积的径流污染产生过程线模型"""
# 2.1 rainfall runoff process simulation
# 初始化
storage = np.zeros((nhour, nus))
runoff = np.zeros((nhour, nus))
storage[0,:] = usloss.T
B = np.zeros((nhour, nus*nlu*npl))
W = np.zeros((nhour, nus*nlu*npl))

# 每个时刻不同下垫面的剩余蓄水能力和径流量动态变化
for t in range(1, nhour):
    for u in range(nus):
        storage[t, u] = min(usloss[u], 
                            storage[t-1, u] - rain[t] + evap[t] + usinfil[u])
        if storage[t, u] < 0:
            # 当蓄水能力小于0时，产生径流
            runoff[t, u] = 0 - storage[t, u]
            storage[t, u] = 0

# 每个时刻各个土地利用类型、下垫面类型的每一种污染物含量动态变化
for t in range(1, nhour):
    for p in range(npl):
        for l in range(nlu):
            for u in range(nus):
                if P[p]["EMC"][u] == 0:
                    if rain[t] == 0:
                        B[t, p*nlu*nus + l*nus + u] = min(P[p]["Bmax"][l], 
                                                                  B[t-1, p*nlu*nus + l*nus + u] + P[p]["bt"][l])
                    else:
                        W[t, p*nlu*nus + l*nus + u] = P[p]["C1"][u] * runoff[t, u]**P[p]["C2"][u] \
                            * B[t-1, p*nlu*nus + l*nus + u]
                        B[t, p*nlu*nus + l*nus + u] = max(0, B[t-1, p*nlu*nus + l*nus + u] \
                            - W[t, p*nlu*nus + l*nus + u])
                else:
                    W[t, p*nlu*nus + l*nus + u] = runoff[t, u] * P[p]["EMC"][u]


## 3. 叠加分析
patchhourlyload = np.zeros((nhour, N*npl))
streethourlyload = np.zeros((nhour, ns*npl))
districthourlyload = np.zeros((nhour, nd*npl))
cityhourlyload = np.zeros((nhour, npl))
totalrunoff = 0 
totalload = np.zeros(npl)

# 遍历所有斑块，以获得不同尺度（斑块、街道、区、城市）每小时的各类污染物的负荷
for n in range(N):
    lu = patchlu[n] - 1 # TODO
    us = patchus[n] -1 # TODO
    # lu = patchlu[n]
    # us = patchus[n]
    totalrunoff += sum(runoff[:, us]) * patcharea[n]
    # 遍历污染指标
    for p in range(npl):
        patchhourlyload[:, n*npl+ p] = W[:,p*nlu*nus + lu*nus + us] * patcharea[n]
        streethourlyload[:, (patchstreet[n]-1)*npl+p] = \
            streethourlyload[:, (patchstreet[n]-1)*npl+p] + patchhourlyload[:, n*npl+ p]
        totalload[p] = totalload[p] + sum(patchhourlyload[:, n*npl+ p])

# 根据每小时的污染物负荷，计算出每天的负荷
patchdailyload = np.zeros((nday, N*npl))
streetdailyload = np.zeros((nday, ns*npl))
districtdailyload = np.zeros((nday, nd*npl))
citydailyload = np.zeros((nday, npl))
for d in range(nday):
    patchdailyload[d, :] = sum(patchhourlyload[d*24:(d+1)*24,:])
    streetdailyload[d, :] = sum(streethourlyload[d*24:(d+1)*24,:])

# 根据每天的污染物负荷，计算出总的负荷
streettotalload = np.zeros((ns, npl))
for s in range(ns):
    for p in range(npl):
        temp = sum(streetdailyload)
        # 计算街道年总负荷
        streettotalload[s, p] = temp[s*npl + p]
        # 计算城市日负荷
        citydailyload[:, p] = citydailyload[:, p] + streetdailyload[:, s*npl + p]

In [1]:
from datetime import datetime
"""4. 总体验证&生成报告"""
save_root = "results/block"
report_dir = "non_point_pollution"
save_dir = os.path.join(save_root,report_dir)
if not os.path.exists(save_dir):
    os.makedirs(save_dir,exist_ok=True)
timestamp = str(datetime.now()).split(".")[0].replace(" ", "T").replace("-","").replace(":","")
save_path = os.path.join(save_dir, "result_rpua_"+timestamp+".xlsx")    

# save
totalrunoff_np = np.zeros((1,1))
totalrunoff_np[0] = totalrunoff
totalrunoff = pd.DataFrame(totalrunoff_np)
totalload = pd.DataFrame(totalload)
runoff = pd.DataFrame(runoff)
W = pd.DataFrame(W)
streettotalload = pd.DataFrame(streettotalload)
citydailyload = pd.DataFrame(citydailyload)

writer = pd.ExcelWriter(save_path)
totalrunoff.to_excel(writer,sheet_name='Sheet1',startrow=1, startcol=1, index=False,header=False)
totalload.to_excel(writer,sheet_name='Sheet1',startrow=2, startcol=1, index=False,header=False)
runoff.to_excel(writer,sheet_name='单位面积径流过程线',startrow=1, startcol=1, index=False,header=False)
W.to_excel(writer,sheet_name='单位面积负荷过程线',startrow=3, startcol=1, index=False,header=False)
streettotalload.to_excel(writer,sheet_name='街道年总负荷分布',startrow=2, startcol=1, index=False,header=False)
citydailyload.to_excel(writer,sheet_name='城市日负荷过程线',startrow=2, startcol=1, index=False,header=False)
writer.save()

NameError: name 'np' is not defined

# 将天气数据从excel转化为txt

In [6]:
import pandas as pd
import numpy as np
import xlrd

file_path = 'data/non_point_pollution/rainfall.xlsx'
io = pd.io.excel.ExcelFile(file_path)

timeseriesdata = pd.read_excel(io).values
# tlabel = [xlrd.xldate.xldate_as_datetime(t,0)
#           for t in timeseriesdata[:, 0]]
rain = timeseriesdata[:, 1]
# evap = timeseriesdata[:, 2]

# data = timeseriesdata[:,:2]
# data = np.zeros((len(rain),3))
# data[:,0] = tlabel
# data[:,1] = rain
# data[:,2] = evap
save_path ="rainfall-20150112T10-20150112T23.txt"
np.savetxt(save_path,rain,delimiter=' ',fmt="%.2f")

In [1]:
import numpy as np
import os 

file_dir = "data/non_point_pollution"
weather_file_path ="rainfall-20150112T10-20150112T23.txt"
file_path = os.path.join(file_dir,weather_file_path)

data = np.loadtxt(file_path)
# rainfall = data[:, 0]  # 降雨量，mm/hr

# 画图

## 降雨量

In [3]:
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# matplotlib.use("Qt5Agg")
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif']=['Microsoft Yahei'] # 设定字体为微软雅黑

class MyFigure(FigureCanvasQTAgg):
   def __init__(self,width=5,height=4,dpi = 100):
      # 1、创建一个绘制窗口Figure对象
      self.fig = Figure(figsize=(width,height),dpi=dpi,tight_layout=True)
      # 2、在父类中激活Figure窗口,同时继承父类属性
      super(MyFigure, self).__init__(self.fig)
 
   # 这里就是绘制图像、示例
   def plotSin(self,x,y):
      self.axes0 = self.fig.add_subplot(111)
      self.axes0.plot(x,y)

def get_xticklabels(start,end,num=6,freq='min'):
   print("freq: ",freq)
   dates = pd.date_range(start=start,end=end,freq=freq)
   date_list = [x.strftime("%Y-%m-%d %H:%M") for x in dates]
   print(date_list)
   show_dates = date_list[::len(dates)//num] + [date_list[-1]]
   
   xticks = list(range(0,len(dates),len(dates)//num)) + [len(dates)-1]
   xlabels = [show_dates[i] for i in range(len(show_dates))]
   
   return xticks, xlabels
 
def fig_rainfall(values, width,height, start_dt_str,end_dt_str, freq):
   F1 = MyFigure(width=width, height=height, dpi=150)
   F1.resize(width,height)
   F1.axes1 = F1.fig.add_subplot(111) 

   fs = 20
   xx_pred = range(len(values))

   F1.axes1.bar(xx_pred, values, width=1,fc='b')
   # plt.show()
   # F1.axes1.plot(xx_pred,values,color='r',lw=2,linestyle="--",label="模拟值")
   xticks, xlabels = get_xticklabels(start_dt_str,end_dt_str, freq=freq)

   # F1.axes1.legend(fontsize=fs-2)
   F1.axes1.set_xlabel(u"时间", fontsize=fs-2)
   F1.axes1.set_ylabel(u"降雨量 (mm/h)", fontsize=fs-5)
   F1.axes1.set_xticks(xticks)
   F1.axes1.set_xticklabels(labels=xlabels,rotation=90)
   # F1.axes1.set_yticklabels(fontsize=fs-2)
   F1.axes1.tick_params(axis='x',labelsize=fs-2)
   F1.axes1.tick_params(axis='y',labelsize=fs)    
   F1.axes1.grid(alpha=0.5,linestyle="-.")
   # F1.show()
   
file_dir = "data/non_point_pollution"
weather_file_path ="rainfall-20150112T10-20150112T23.txt"
file_path = os.path.join(file_dir,weather_file_path)

data = np.loadtxt(file_path)

fig_rainfall(values=data, width=520,height=360, 
             start_dt_str="2022-09-14T10",
             end_dt_str="2022-09-14T23", 
             freq="1H")


freq:  1H
['2022-09-14 10:00', '2022-09-14 11:00', '2022-09-14 12:00', '2022-09-14 13:00', '2022-09-14 14:00', '2022-09-14 15:00', '2022-09-14 16:00', '2022-09-14 17:00', '2022-09-14 18:00', '2022-09-14 19:00', '2022-09-14 20:00', '2022-09-14 21:00', '2022-09-14 22:00', '2022-09-14 23:00']


: 